In [1]:
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
import janitor

In [2]:
train = pd.read_csv("../data/train_new.csv")
test = pd.read_csv("../data/test_new.csv")

In [3]:
train

,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,target
0,L,298.3,309.1,1616,31.1,195,0
1,L,298.2,308.4,1388,53.8,137,0
2,L,298.2,307.8,1528,31.1,194,0
3,M,300.9,310.8,1599,33.0,7,0
4,L,301.4,310.5,1571,33.9,208,0
...,...,...,...,...,...,...,...
7995,L,302.3,311.8,1369,56.2,208,1
7996,L,304.0,313.2,1416,46.0,128,0
7997,H,302.8,312.3,1483,47.2,223,0
7998,H,296.1,306.9,1541,32.6,33,0


In [4]:
#train = train.drop(["cabin", "name", "passengerid"], axis=1)
#test = test.drop(["cabin", "name", "passengerid"], axis = 1)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   type                   8000 non-null   object 
 1   air_temperature_k      8000 non-null   float64
 2   process_temperature_k  8000 non-null   float64
 3   rotational_speed_rpm   8000 non-null   int64  
 4   torque_nm              8000 non-null   float64
 5   tool_wear_min          8000 non-null   int64  
 6   target                 8000 non-null   int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 437.6+ KB


In [6]:
train["type"] = train["type"].astype("category")

In [7]:
cat_col = []
num_col = []
for col in list(train.columns):
    if is_categorical_dtype(train[col]):
        cat_col.append(col)
    elif is_numeric_dtype(train[col]):
        num_col.append(col)

In [8]:
num_col, len(num_col)

(['air_temperature_k',
  'process_temperature_k',
  'rotational_speed_rpm',
  'torque_nm',
  'tool_wear_min',
  'target'],
 6)

In [9]:
num_col.remove("target")

In [10]:
cat_col

['type']

In [11]:
train

,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,target
0,L,298.3,309.1,1616,31.1,195,0
1,L,298.2,308.4,1388,53.8,137,0
2,L,298.2,307.8,1528,31.1,194,0
3,M,300.9,310.8,1599,33.0,7,0
4,L,301.4,310.5,1571,33.9,208,0
...,...,...,...,...,...,...,...
7995,L,302.3,311.8,1369,56.2,208,1
7996,L,304.0,313.2,1416,46.0,128,0
7997,H,302.8,312.3,1483,47.2,223,0
7998,H,296.1,306.9,1541,32.6,33,0


In [12]:
ordi = OrdinalEncoder(categories=[["L", "M", "H"]])
minmax = MinMaxScaler()
onehot = OneHotEncoder()

In [13]:
#all_col = num_col.copy()
#all_col.append("type")

In [14]:
ct = ColumnTransformer([("ord", ordi, ["type"]),
                        ("minmax", minmax, num_col)], remainder="passthrough")

In [15]:
#new_ct = ColumnTransformer([("minmax", minmax, ["type")], remainder="passthrough")

In [16]:
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('ord',
                                 OrdinalEncoder(categories=[['L', 'M', 'H']]),
                                 ['type']),
                                ('minmax', MinMaxScaler(),
                                 ['air_temperature_k', 'process_temperature_k',
                                  'rotational_speed_rpm', 'torque_nm',
                                  'tool_wear_min'])])

In [17]:
train_use = ct.transform(train)
new_features = list(ct.get_feature_names_out())
for i in range(len(new_features)):
    new_features[i] = new_features[i].split("__")[1]
train_df = pd.DataFrame(train_use, columns= new_features)
train_df["type"] = minmax.fit_transform(np.array(train_df["type"]).reshape(-1,1))
train_df = train_df.clean_names()

In [18]:
test_use = ct.transform(test)
#new_features = list(ct.get_feature_names_out())
test_df = pd.DataFrame(test_use, columns=new_features)
test_df["type"] = minmax.fit_transform(np.array(test_df["type"]).reshape(-1,1))

In [19]:
test_df = test_df.clean_names()

In [20]:
train_df.columns

Index(['type', 'air_temperature_k', 'process_temperature_k',
       'rotational_speed_rpm', 'torque_nm', 'tool_wear_min', 'target'],
      dtype='object')

In [21]:
train_df.to_csv("../data/train_fe.csv", index=False)
test_df.to_csv("../data/test_fe.csv", index=False)